# The following are analysis performed for this Bachelor Thesis

In [66]:
%load_ext autoreload
%autoreload 2

from tabulate import tabulate
from texttable import Texttable
import latextable
import header as head
import json
import os

annotations_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations"
community_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection"
bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"
figure_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Figures"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


There are 74 rumour source tweet with 0 non-rumour source tweet within the total of 74 rumour tweet.

In [ ]:
# p = head.getPath()
# print(p)
head.count()

In [ ]:
# merging following relationship into a JSON file
head.followRelate()

with open('/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json') as f:
    data = json.load(f)
    followRelate_Ids = list(data.keys())

In [ ]:
# generate dictionary for tweetId and userId conversion
# results are saved in Id-conversions folder
head.gen_idConversion()

There are followers who reacted to the source tweets. 
* reacts from followers are saved in `react_follows`
* reacts from non-followers are saved in `react_not_follows`

In [ ]:
# See whether there are any reactors that follows the person they responded to 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json") as f:
    followingRelate = json.load(f)

head.is_follower_react(followingRelate)

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    who_follows_whom = json.load(file)

bridges = communities["bridges"]
rumours = communities["rumours"]
non_rumours = communities["non_rumours"]

bridges_follow = {}

for bridge in bridges:
    follows = who_follows_whom[bridge]
    bridges_follow[bridge] = {
        "rumours":[],
        "non_rumours":[],
        "bridges":[]
    }

    for f in follows:
        if f in rumours: (bridges_follow[bridge])["rumours"].append(f)
        elif f in non_rumours: (bridges_follow[bridge])["non_rumours"].append(f)
        elif f in bridges: (bridges_follow[bridge])["bridges"].append(f)

head.writeToJSON(head.getRoot(),"bridges_follows",bridges_follow)


### Reacting Relationship

In [67]:
head.who_reacted_by_whom()

In [68]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_reacted_by_whom.json") as file:
    reacted_by = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

rumours = communities["rumours"]
non_rumours = communities["non_rumours"]
bridges = communities["bridges"]
uncategorized = communities["uncategorized"]

cat_who_reacted_by_whom = {}

for id in reacted_by:
    cat_who_reacted_by_whom[id] = {}
    user = cat_who_reacted_by_whom[id]
    user["rumours"] = list(set(reacted_by[id]).intersection(set(rumours)))
    user["non_rumours"] = list(set(reacted_by[id]).intersection(set(non_rumours)))
    user["bridges"] = list(set(reacted_by[id]).intersection(set(bridges)))
    user["uncategorized"] = list(set(reacted_by[id]).intersection(set(uncategorized)))

    if not(user["rumours"] or user["non_rumours"] or user["bridges"] or user["uncategorized"]):
        cat_who_reacted_by_whom.pop(id)

head.writeToJSON(head.getRoot(),"who_reacted_by_whom_2",cat_who_reacted_by_whom)

### Tabular

In [28]:
# follow + follower counts
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "m", "b"])
table.add_rows([["id","#rumour followers","#non-rumour followers"]])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = [id,bridge["rum_count"],bridge["nonrum_count"]]
    data.append(info)

table.add_rows(data)
with open('follower_counts.txt', 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))



In [19]:
# follow + betweeness + centrality
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality_2.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "t", "t"])
table.set_cols_dtype(["i","e","e"])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = []
    if bridge["rumours"]:
        info.append([id,"betweeness","centrality"])
        info.append(["rumours","",""])
        
        for follower in bridge["rumours"]:
            user = (bridge["rumours"])[follower]
            info.append([follower,user["betweeness"],user["closeness"]])
    
    elif bridge["non_rumours"]:
        info.append([id,"betweeness","centrality"])
        info.append(["non-rumours","",""])

        for follower in bridge["non_rumours"]:
            user = (bridge["non_rumours"])[follower]
            info.append([follower,user["betweeness"],user["closeness"]])
    
    data.extend(info)

table.add_rows(data)
# print(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))
with open(head.makePath([figure_path,"follow","follower_centrality.txt"]), 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))

